In [1]:
'''
Application 主架構
'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson = json.load(open('./line_secret_key', 'r', encoding='utf-8'))
server_url = secretFileContentJson.get('server_url')
channel_access_token = secretFileContentJson["channel_access_token"]


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

#生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get('channel_access_token'))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route('/', methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [2]:
'''
消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。
'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_message_array(fileName):
    
    #打開檔案轉成json格式
    with open(fileName, encoding='utf-8') as f:
        jsonArray = json.load(f)
    
    #解析json
    returnArray = []
    for jsonObject in jsonArray:
        
        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'carouselTemplate':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray
        
    
    
        

In [3]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)



uriQuickReplyButton = QuickReplyButton(
    action=URIAction(
        label="西甲", 
        uri="line://app/1614188410-mgw5LXGz"
    )
)

textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="line://app/1614188410-mgw5LXGz"
    )
)

In [4]:
quickReplyList = QuickReply(
    items = [textQuickReplyButton, uriQuickReplyButton]
)

In [5]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage
)
quickReplyTextSendMessage = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [6]:
'''
將選單綁定到特定用戶上

取得rich_menu_id
取得self_user_id
設定line的遠端位置
設定消息的基本安全驗證
發送消息告知

'''
# #取得rich_menu_id
# link_rich_menu_id = json.load(open('./material/rich_menu/rich_menu_main/rich_menu_id.json', 'r', encoding='utf-8'))
# link_rich_menu_id = link_rich_menu_id['richMenuId']

# #取得self_user_id
# self_user_id = json.load(open('./line_secret_key', 'r', encoding='utf-8'))
# self_user_id = self_user_id['self_user_id']

# # 將菜單id與用戶id組合成遠端位置
# linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (self_user_id, link_rich_menu_id)

# # 設定消息基本安全憑證
# linkMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# # 發送消息
# lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
# print(lineLinkMenuResponse)
# print(lineLinkMenuResponse.text)

'\n將選單綁定到特定用戶上\n\n取得rich_menu_id\n取得self_user_id\n設定line的遠端位置\n設定消息的基本安全驗證\n發送消息告知\n\n'

In [7]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate,CarouselTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='Adidas  F50 FG(2015)',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          }
        ],
)
)

In [8]:
import json
def create_carousel_template_bott(replyJsonPath): 
    replyJson = json.load(open(replyJsonPath, 'r', encoding='utf-8'))
    cols = replyJson['columns']
    Carousel_template = TemplateSendMessage(
        alt_text='Carousel template',
        template=CarouselTemplate(
            columns=[
                CarouselColumn(
                    thumbnail_image_url=replyJson['columns'][0]['thumbnailImageUrl'],
                    title=replyJson['columns'][0]['title'],
                    text=replyJson['columns'][0]['text'],
                    actions=[
                        MessageTemplateAction(
                            label=replyJson['columns'][0]['action'][0]['label'],
                            text=replyJson['columns'][0]['action'][0]['text'],
                        ),
                        URITemplateAction(
                            label=replyJson['columns'][0]['action'][1]['label'],
                            uri=replyJson['columns'][0]['action'][1]['uri']
                        )
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url=replyJson['columns'][1]['thumbnailImageUrl'],
                    title=replyJson['columns'][1]['title'],
                    text=replyJson['columns'][1]['text'],
                    actions=[
                        MessageTemplateAction(
                            label=replyJson['columns'][1]['action'][0]['label'],
                            text=replyJson['columns'][1]['action'][0]['text'],
                        ),
                        URITemplateAction(
                            label=replyJson['columns'][1]['action'][1]['label'],
                            uri=replyJson['columns'][1]['action'][1]['uri']
                        )
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url=replyJson['columns'][2]['thumbnailImageUrl'],
                    title=replyJson['columns'][2]['title'],
                    text=replyJson['columns'][2]['text'],
                    actions=[
                        MessageTemplateAction(
                            label=replyJson['columns'][2]['action'][0]['label'],
                            text=replyJson['columns'][2]['action'][0]['text'],
                        ),
                        URITemplateAction(
                            label=replyJson['columns'][2]['action'][1]['label'],
                            uri=replyJson['columns'][2]['action'][1]['uri']
                        )
                    ]
                )
            ]
        )
    )

In [9]:
# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    result_message_array  = []
    replyJsonPath = './material/follow/reply.json'
    result_message_array = detect_json_array_to_message_array(replyJsonPath)
    
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

In [10]:
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs

@handler.add(PostbackEvent)
def process_post_back_event(event):

    query_string_list = parse_qs(event.postback.data)
    print(query_string_list)
    if 'change_menu' in query_string_list:
        
        #將用戶與另一張圖文選單綁定
        Path = './material/rich_menu/rich_menu_' + query_string_list['change_menu'][0]
        linkRichMenuId = json.load(open(Path + '/rich_menu_id.json', 'r', encoding='utf-8'))['richMenuId']
        line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)
        
        #回傳訊息
        replyJsonPath = './material/rich_menu/' + query_string_list['change_menu'][0] + '_reply'
        result_message_array = detect_json_array_to_message_array(replyJsonPath + '/reply.json')
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'quick_reply' in query_string_list:
        replyJsonPath = './material/rich_menu/rich_menu_' + query_string_list['change_menu']
        print(query_string_list)
        print(query_string_list['quick_reply'])
    elif 'shoes_type' in query_string_list:
        replyJsonPath = './material/rich_menu/shoes_reply/' + query_string_list['shoes_type'][0]
        result_message_array = detect_json_array_to_message_array(replyJsonPath + '/reply.json')
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        #重新綁定主要圖文選單
        Path = './material/rich_menu/rich_menu_main'
        linkRichMenuId = json.load(open(Path + '/rich_menu_id.json', 'r', encoding='utf-8'))['richMenuId']
        line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)
    elif 'schedule_league' in query_string_list:
        print(query_string_list)
        replyJsonPath = './material/rich_menu/schedule_reply/' + query_string_list['schedule_league'][0]
        print(replyJsonPath)
        result_message_array = detect_json_array_to_message_array(replyJsonPath + '/reply.json')
        print(result_message_array)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        Path = './material/rich_menu/rich_menu_main'
        linkRichMenuId = json.load(open(Path + '/rich_menu_id.json', 'r', encoding='utf-8'))['richMenuId']
        line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)


In [11]:
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# @handler.add(MessageEvent, message=TextMessage)
# def handle_message(event):
#     line_bot_api.reply_message(
#         event.reply_token,
#         quickReplyTextSendMessage
#     )


In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Sep/2019 16:02:01] "POST / HTTP/1.1" 200 -


{'change_menu': ['schedule']}


[2019-09-27 16:02:03,191] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1935, in d

{'schedule_league': ['spain']}
{'schedule_league': ['spain']}
./material/rich_menu/schedule_reply/spain
[{"text": "\u897f\u7532\u8cfd\u7a0b: \nline://app/1614188410-kZnJDzqe", "type": "text"}, {"text": "\u5982\u679c\u9700\u8981\u5176\u4ed6\u670d\u52d9\u8acb\u91cd\u65b0\u9ede\u9078\u9078\u55ae0.<", "type": "text"}]


...

127.0.0.1 - - [27/Sep/2019 16:02:32] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2019 16:02:33] "POST / HTTP/1.1" 200 -
[2019-09-27 16:02:35,195] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\user\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispat

{'change_menu': ['schedule']}


127.0.0.1 - - [27/Sep/2019 16:02:36] "POST / HTTP/1.1" 200 -


{'schedule_league': ['france']}
{'schedule_league': ['france']}
./material/rich_menu/schedule_reply/france
[{"text": "\u6cd5\u7532\u8cfd\u7a0b: \nline://app/1614188410-aEnoM1Jm", "type": "text"}, {"text": "\u5982\u679c\u9700\u8981\u5176\u4ed6\u670d\u52d9\u8acb\u91cd\u65b0\u9ede\u9078\u9078\u55ae0.<", "type": "text"}]


127.0.0.1 - - [27/Sep/2019 16:02:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2019 16:02:38] "POST / HTTP/1.1" 200 -
